## Scenario 2: A cross-functional team with one data scientist working on an ML model


MLflow setup:
- tracking server: yes, local server
- backend store: sqlite database
- artifacts store: local filesystem

The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

`mlflow server --backend-store-uri sqlite:///backend.db`

In [2]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [3]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [4]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716474818070, experiment_id='0', last_update_time=1716474818070, lifecycle_stage='active', name='Default', tags={}>]

In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/23 14:36:54 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: '/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/6fab6c78ea134932949201f7e08b81f3/artifacts'


In [6]:
mlflow.search_experiments()

[<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1', creation_time=1716475014755, experiment_id='1', last_update_time=1716475014755, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716474818070, experiment_id='0', last_update_time=1716474818070, lifecycle_stage='active', name='Default', tags={}>]

### Interacting with the model registry

In [7]:
from mlflow.tracking import MlflowClient


client = MlflowClient("http://127.0.0.1:5000")

In [10]:
client.search_registered_models()

[]

In [16]:
client.search_runs(experiment_ids='6fab6c78ea134932949201f7e08b81f3')


[]

In [21]:
mlflow.search_runs(search_all_experiments=True)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.accuracy,params.C,params.random_state,tags.mlflow.log-model.history,tags.mlflow.source.name,tags.mlflow.source.type,tags.mlflow.user,tags.mlflow.runName
0,6fab6c78ea134932949201f7e08b81f3,1,FINISHED,/workspaces/mlops-zoomcamp/02-experiment-track...,2024-05-23 14:36:54.838000+00:00,2024-05-23 14:36:56.766000+00:00,0.96,0.1,42,"[{""run_id"": ""6fab6c78ea134932949201f7e08b81f3""...",/home/codespace/anaconda3/envs/exp-tracking/li...,LOCAL,codespace,respected-shad-766


In [29]:
run_id = mlflow.search_runs(experiment_names=['my-experiment-1'])['run_id'][0]
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/23 15:01:36 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716476496754, current_stage='None', description='', last_updated_timestamp=1716476496754, name='iris-classifier', run_id='6fab6c78ea134932949201f7e08b81f3', run_link='', source='/workspaces/mlops-zoomcamp/02-experiment-tracking/running-mlflow-examples/artifacts_local/1/6fab6c78ea134932949201f7e08b81f3/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>